<a href="https://colab.research.google.com/github/Alex-Witt/DS-Unit-1-Sprint-1-Dealing-With-Data/blob/master/module1-afirstlookatdata/LS_DS_111_A_First_Look_at_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science - A First Look at Data



## Lecture - let's explore Python DS libraries and examples!

The Python Data Science ecosystem is huge. You've seen some of the big pieces - pandas, scikit-learn, matplotlib. What parts do you want to see more of?

In [0]:
# TODO - we'll be doing this live, taking requests
# and reproducing what it is to look up and learn things

1 + 1
print("This is a Change")

This is a Change


## Assignment - now it's your turn

Pick at least one Python DS library, and using documentation/examples reproduce in this notebook something cool. It's OK if you don't fully understand it or get it 100% working, but do put in effort and look things up.

The Notebook I decided on was the **SF Drug Geography** by Lance Martin. 

**Original Notebook**: https://github.com/lmart999/GIS/blob/master/SF_GIS_Crime.ipynb

**Data Set Location**: https://data.sfgov.org/Public-Safety/Police-Department-Incident-Reports-Historical-2003/tmnf-yvry

The dataset that I am using is an updated version with far more entries in it. First I will repeat some of the broad Crime Visualizations Lance put together now that there is more data at my disposal. 

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
df = pd.read_csv("https://www.dropbox.com/s/v1s4hhj2hlghr99/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv?dl=1")
df.shape

In [0]:
df.head(5)

It appears we have roughly 400,000 more rows than the previous dataset. The first problem I start running into as an inexperienced programmer, is that the dataset is taking a while to do things. This is not good for me, as I need to constantly test my code to catch errors. So I am going to utilize the Train Test Split function I last used in the intro course to test my code more quickly. 

In [0]:
from sklearn.model_selection import train_test_split
df_x, df_y = train_test_split(df.copy(), random_state = 0)
df_y.shape

~ 500,000 is easier to deal with. I also want to create a single Row Dataframe just so I can test something fast. 

In [0]:
##Create single row dataframe

In [0]:
##Date manipulation. I may not need anymore. 

date=pd.to_datetime(df_y['Date'])

# To Be Continued!

### Assignment questions

After you've worked on some code, answer the following questions in this text block:

1.  Describe in a paragraph of text what you did and why, as if you were writing an email to somebody interested but nontechnical.

2.  What was the most challenging part of what you did?

3.  What was the most interesting thing you learned?

4.  What area would you like to explore with more time?




## Stretch goals and resources

Following are *optional* things for you to take a look at. Focus on the above assignment first, and make sure to commit and push your changes to GitHub (and since this is the first assignment of the sprint, open a PR as well).

- [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/)
- [scikit-learn documentation](http://scikit-learn.org/stable/documentation.html)
- [matplotlib documentation](https://matplotlib.org/contents.html)
- [Awesome Data Science](https://github.com/bulutyazilim/awesome-datascience) - a list of many types of DS resources

Stretch goals:

- Find and read blogs, walkthroughs, and other examples of people working through cool things with data science - and share with your classmates!
- Write a blog post (Medium is a popular place to publish) introducing yourself as somebody learning data science, and talking about what you've learned already and what you're excited to learn more about.